<a href="https://colab.research.google.com/github/Qi-Li3/DS3500-Final-Project/blob/main/starbucks_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print(5)